In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import src.feature_preprocess as feature_preprocess
from sklearn.decomposition import LatentDirichletAllocation as LDA
import pyLDAvis
import pyLDAvis.sklearn

/Users/sergey_kuznetsov/.pyenv/versions/3.9.4/envs/hakathon_hyper_ad_tech_venv/lib/python3.9/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


# Clasterisation

In [2]:
train = pd.read_csv("data/train.csv")
print(train.shape)

(44854516, 10)


У нас очень много строчек - LDA будет работать слишком долго, поэтому оставим только самые лучшие строчки(по сути найденные кластеры должны быть глобальными,поэтому выкидывание строчек не должно быть проблемой)

Только bundle id андроид имеют говорящие названия - поэтому оставим только их

In [3]:
train['os'] = train['os'].str.upper()
train = train[train['os'] == 'ANDROID']

Удалим строчки с пропусками и дубли.

In [ ]:
train = train.dropna()
train = train.drop_duplicates() 
train.shape

(20918176, 10)

Все еще слишком много - возьмем подмножество строчек.

In [ ]:
train = train.sample(1000000)

In [ ]:
train.to_csv('data/train_na.csv')

In [2]:
train = pd.read_csv('data/train_na.csv')

Теперь добавим интерпретируемых тэгов

In [3]:
train = train.reset_index(drop=True)

time_features = feature_preprocess.make_features_from_time(train)
time_tags = feature_preprocess.get_tags_from_time_features(time_features)
time_tags = time_tags.rename(columns={'loc_weekday_tag': 'weekday', 'loc_is_weekend_tag': 'weekend', 'loc_is_academic_year_tag': 'учебный год', 'loc_time_of_day_tag': 'время'})
time_tags = time_tags.drop(columns='weekday')
print('time_tags finished')

city_features = feature_preprocess.make_features_from_cities(train, "data/data_cities.csv")
city_tags = feature_preprocess.get_tags_from_cities_features(city_features)
print('city_tags finished')

phone_tags = feature_preprocess.phone_tags(train)
print('phone_tags finished')

train = pd.concat([train, time_tags, city_tags, phone_tags], axis=1)

train = train.rename(columns={'gamecategory': 'Кат', 'subgamecategory': 'ПодКат'})

train = train.dropna()

time_tags finished
city_tags finished


/Users/sergey_kuznetsov/Documents/hackathon_hyper_ad_tech/src/feature_preprocess.py:317: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out['os'] = out['os'].str.upper()
/Users/sergey_kuznetsov/Documents/hackathon_hyper_ad_tech/src/feature_preprocess.py:318: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out['osv_num'] = data['osv'].astype(str).apply(get_version_float)
/Users/sergey_kuznetsov/Documents/hackathon_hyper_ad_tech/src/feature_preprocess.py:319: SettingWithCopyWarning: 
A value is trying to be set 

phone_tags finished


Теперь соберем бандл и тэги в description

In [4]:
description = feature_preprocess.create_description(train, ['bundle', 'Кат', 'ПодКат'] + list(time_tags.columns) + list(city_tags.columns) + list(phone_tags.columns))
description = description.drop_duplicates()
description

Кат
(807608,)
ПодКат
(807608,)
weekend
(807608,)
учебный год
(807608,)
время
(807608,)
type_city
(807608,)
size_city
(807608,)
timezone
(807608,)
os_
(807608,)
new_phone
(807608,)


1         com.supercell.hayday.tag: Games tag_t: Кат.tag...
2         es.socialpoint.MonsterLegends.tag: Games tag_t...
3         me.zepeto.main.tag: Applications tag_t: Кат.ta...
4         com.merge.cube.winner.tag: Games tag_t: Кат.ta...
5         com.inspiredsquare.blocks.tag: Games tag_t: Ка...
                                ...                        
999957    com.bigcool.puzzle.candygenies.tag: Games tag_...
999966    com.feofun.elementalmaster.tag: Games tag_t: К...
999979    sort.water.puzzle.pour.color.tubes.sorting.gam...
999984    com.dunderbit.humbug.tag: Games tag_t: Кат.tag...
999989    com.infraware.office.link.tag: Applications ta...
Length: 290133, dtype: object

In [5]:
count_vect = CountVectorizer(tokenizer=lambda x: x.split('.'),
                             strip_accents = 'unicode',
                             lowercase = True)

alpha = 1 
beta = 0.05

lda = LDA(n_components=7,
        #   learning_method='online',
          doc_topic_prior = alpha,
          topic_word_prior = beta,
          n_jobs=-1,
          random_state=12345,
          verbose=True)

matrix_count_vec = count_vect.fit_transform(description.astype(str))

lda.fit(matrix_count_vec)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, matrix_count_vec, count_vect)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


/Users/sergey_kuznetsov/.pyenv/versions/3.9.4/envs/hakathon_hyper_ad_tech_venv/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/Users/sergey_kuznetsov/.pyenv/versions/3.9.4/envs/hakathon_hyper_ad_tech_venv/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/sergey_kuznetsov/.pyenv/versions/3.9.4/envs/hakathon_hyper_ad_tech_venv/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/sergey_kuznetsov/.pyenv/versions/3.9.4/env

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.058292  0.149471       1        1  14.814221
0      0.053522 -0.066552       2        1  14.680670
1     -0.042229 -0.057071       3        1  14.664660
2      0.037541 -0.076809       4        1  14.466771
4      0.050227  0.019126       5        1  14.190176
6      0.013140 -0.001147       6        1  14.081908
5     -0.170493  0.032982       7        1  13.101594, topic_info=                                      Term          Freq         Total  \
13866       tag: big city tag_t: size_city  90835.000000  90835.000000   
13858            tag: action tag_t: подкат  34280.000000  34280.000000   
13861         tag: applications tag_t: кат  29817.000000  29817.000000   
13912  tag: very big city tag_t: size_city  54519.000000  54519.000000   
13900            tag: puzzle tag_t: подкат  46597.000000  46597.000000   
...                                    ...           ...           ...   
13905     tag: small city tag_t: size_city   5594.884174  31876.492838   
13933        tag: раннее_утро tag_t: время   5321.883240  29466.518695   
13916              tag: вечер tag_t: время   5905.322017  47963.351717   
13926               tag: ночь tag_t: время   4787.945765  21447.157160   
13931      tag: позднии_вечер tag_t: время   4584.005997  35474.402290   

      Category  logprob  loglift  
13866  Default  30.0000  30.0000  
13858  Default  29.0000  29.0000  
13861  Default  28.0000  28.0000  
13912  Default  27.0000  27.0000  
13900  Default  26.0000  26.0000  
...        ...      ...      ...  
13905   Topic7  -4.5361   0.2924  
13933   Topic7  -4.5861   0.3210  
13916   Topic7  -4.4821  -0.0621  
13926   Topic7  -4.6918   0.5329  
13931   Topic7  -4.7353  -0.0138  

[530 rows x 6 columns], token_table=       Topic      Freq       Term
term                             
27         1  0.997298   aastudio
27         7  0.001401   aastudio
118        4  0.999764  adventure
185        1  0.999837        air
329        4  0.998915   amanotes
...      ...       ...        ...
16192      3  0.003782      zynga
16192      4  0.001261      zynga
16192      5  0.003782      zynga
16192      6  0.001261      zynga
16192      7  0.002522      zynga

[858 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3, 5, 7, 6])

Посмотрим на покрытие